In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import string
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import gensim.downloader

In [ ]:
word_vectors = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


In [ ]:
# Mounting the gdrive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Opening training files.
file_path = "/content/gdrive/MyDrive/Brown_POS Dataset/Brown_tagged_train.txt"
# json_path = "/content/a03_test.txt"

file = open(file_path,'r')
text = file.read()
file.close()


In [ ]:
def get_data(path):
    
    file = open(file_path,'r')
    text = file.read()
    file.close()

    sent = []
    for line in text.split('\n'):
        sent.append(line)

    text = []
    labels = []

    for sentence in sent:
        sentence = sentence.strip()
        words = sentence.split(" ")
        
        for word in words:
            s_index = word.rfind("/")
            sen_word = (word[:s_index]).lower()
            label = word[s_index + 1:]
            text.append(sen_word)
            labels.append(label)
            # print(word, label)
    
    return text, labels

In [ ]:
# Separating the words and labels
words, tags = get_data(file_path)
words = [words]
labels = set(tags)

# print(labels)
# word_list = {}


# for tag in tags:
#     if tag in word_list.keys():
#         word_list[tag] += 1

#     else:
#         word_list[tag] = 1
    

# print(word_list)

In [ ]:
# Label Encoder
le = preprocessing.LabelEncoder()
y = le.fit_transform(tags)
y = to_categorical(y, len(labels))

In [ ]:
def checkWord(word):
    tokenizer = RegexpTokenizer('\w+')
    words = tokenizer.tokenize(word)
    return words



In [ ]:
m,n = np.shape(words)
X_w2v = []
for i in range(n):
    try:
        vec = word_vectors.wv[words[0][i]]
    except KeyError:
         vec = np.random.randn(300)*(0.01)

    X_w2v.append(vec)

feature_len = len(X_w2v[0])
num_classes = len(labels)
input_shape = (feature_len,)
y = np.array(y)
X_w2v = np.array(X_w2v)


In [ ]:
# Making the model
model = Sequential()
model.add(Dense(350, input_shape = input_shape, activation='relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_w2v, y, test_size = 0.20)

In [ ]:
# Training the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 10, batch_size = 200)
test_results = model.evaluate(X_test, y_test)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

# y_pred = model.predict(X_test)
# y_pred = np.argmax(y_pred, axis = 1)
# y_pred = le.inverse_transform(y_pred)
# y_pred_dict = {}



In [ ]:
# y_test_new = le.inverse_transform(y_test)
# red_word_list = {}
# for i in range(len(y_test)):
#     if (y_pred[i] != y_test_new[i]):
#         print("yes")
#         if (y_test_new[i] in red_word_list.keys()):
#             red_word_list[y_test_new[i]] += 1
#         else:
#             red_word_list[y_test_new[i]] = 1

        


In [ ]:
# # red_word_list = word_list
# print(red_word_list)
# print(word_list)
# print(y_pred)

# toPlot = {}
# for key, value in word_list.items():
#     val = red_word_list.get(key)
#     if val:
#         toPlot[key] = val/value
#     else:
#         toPlot[key] = 0

# print(toPlot)


In [ ]:
# plt.bar(range(len(toPlot)), list(toPlot.values()), align='center', width = 0.4)
# plt.xticks(range(len(toPlot)), list(toPlot.keys()))

# plt.show()

In [ ]:
# Loss function without one hot encoding
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 20, batch_size = 200)
test_results = model.evaluate(X_test, y_test)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')


In [ ]:
# Glove
def checkWord(word):
    tokenizer = RegexpTokenizer('\w+')
    words = tokenizer.tokenize(word)
    return words


embeddings_index = dict()
f = open('/content/glove.6B.300d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
model_g = []
for i in words[0]:
    if embeddings_index.get(i) is not None:
        model_g.append(embeddings_index.get(i))
    else:

        words_new = checkWord(i)
        val = [1]*(300)
        
        for each_word in words_new:
            if embeddings_index.get(each_word) is not None:
                val = np.multiply(val, embeddings_index.get(each_word))

        if (max(val) == 1 and min(val) == 1):
            val = np.random.randn(300)*(0.01)

        model_g.append(val)

y = np.array(y)
X_g = np.array(model_g)

feature_len = len(X_g[0])
input_shape = (feature_len,)
        


In [ ]:
# Label Encoder
le = preprocessing.LabelEncoder()
y = le.fit_transform(tags)
y = to_categorical(y, len(labels) )

num_classes = len(labels)




In [ ]:
model = Sequential()
model.add(Dense(350, input_shape = input_shape, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation = 'softmax'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_g, y, test_size = 0.20)

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 10, batch_size = 200, verbose = 1)
test_results = model.evaluate(X_test, y_test, verbose = 1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')


# y_pred = model.predict(X_test)
# y_pred = np.argmax(y_pred, axis = 1)
# y_pred = le.inverse_transform(y_pred)

        


In [ ]:
# y_test_new = np.argmax(y_test, axis = 1)
# y_test_new = le.inverse_transform(y_test_new)
# red_word_list = {}
# for i in range(len(y_test_new)):
#     if (y_test_new[i] != y_pred[i]):
#         if (y_test_new[i] in red_word_list.keys()):
#             red_word_list[y_test_new[i]] += 1
#         else:
#             red_word_list[y_test_new[i]] = 1


In [ ]:
# # red_word_list = word_list
# print(red_word_list)
# print(word_list)
# print(y_pred)

# toPlot = {}
# for key, value in word_list.items():
#     val = red_word_list.get(key)
#     if val:
#         toPlot[key] = val/value
#     else:
#         toPlot[key] = 0

# print(toPlot)


In [ ]:
# plt.bar(range(len(toPlot)), list(toPlot.values()), align='center', width = 0.4)
# plt.xticks(range(len(toPlot)), list(toPlot.keys()))

# plt.show()

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 20, batch_size = 200)
test_results = model.evaluate(X_test, y_test)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')


In [ ]:
# Epochs for glove and word2vec
acc_w2v = []
acc_g = []

num_epochs = [5, 10, 15, 20, 30]
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size = 0.20)
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g, y, test_size = 0.20)

model = Sequential()
model.add(Dense(350, input_shape = input_shape, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation = 'softmax'))

for i in range(len(num_epochs)):

    model = Sequential()
    model.add(Dense(350, input_shape = input_shape, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation = 'softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train_w2v, y_train_w2v, epochs = num_epochs[i], batch_size = 200)
    test_results = model.evaluate(X_test_w2v, y_test_w2v)
    acc_w2v.append(test_results[1])

    model = Sequential()
    model.add(Dense(350, input_shape = input_shape, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train_g, y_train_g, epochs = num_epochs[i], batch_size = 200)
    test_results = model.evaluate(X_test_g, y_test_g)
    acc_g.append(test_results[1])







In [ ]:
plt.plot(num_epochs, acc_w2v, color = "red", label = "Word 2 Vec")
plt.plot(num_epochs, acc_g, color = "blue", label = "Glove")
plt.title("Accuracy with epochs")
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
# Activation Functions for glove and word2vec
acc_w2v = []
acc_g = []

act_funcs = ["relu", "sigmoid", "tanh", "selu"]
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size = 0.20)
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g, y, test_size = 0.20)


for i in range(len(act_funcs)):
        
    model = Sequential()
    model.add(Dense(350, input_shape = input_shape, activation = act_funcs[i]))
    model.add(Dense(50, activation = act_funcs[i]))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    
    # Word 2 vec
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train_w2v, y_train_w2v, epochs = 30, batch_size = 200)
    test_results = model.evaluate(X_test_w2v, y_test_w2v)
    acc_w2v.append(test_results[1])

    
    model = Sequential()
    model.add(Dense(350, input_shape = input_shape, activation = act_funcs[i]))
    model.add(Dense(50, activation = act_funcs[i]))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    # Glove
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train_g, y_train_g, epochs = 30, batch_size = 200)
    test_results = model.evaluate(X_test_g, y_test_g)
    acc_g.append(test_results[1])






In [ ]:

plt.plot(act_funcs, acc_w2v, color = "red", label = "Word 2 Vec")
plt.plot(act_funcs, acc_g, color = "blue", label = "Glove")
plt.title("Accuracy with activation functions")
plt.xlabel("Activation functions")
plt.ylabel("Accuracy")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
# Optimizers for glove and word2vec
acc_w2v = []
acc_g = []

opt = ["adam", "RMSprop", "sgd", "adagrad"]
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size = 0.20)
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g, y, test_size = 0.20)



for i in range(len(opt)):
    
    model = Sequential()
    model.add(Dense(350, input_shape = input_shape, activation = "relu"))
    model.add(Dense(50, activation = "relu" ))
    model.add(Dense(num_classes, activation = 'softmax'))
        
    
    
    # Word 2 vec
    model.compile(loss = 'categorical_crossentropy', optimizer = opt[i] , metrics = ['accuracy'])
    model.fit(X_train_w2v, y_train_w2v, epochs = 15, batch_size = 200)
    test_results = model.evaluate(X_test_w2v, y_test_w2v)
    acc_w2v.append(test_results[1])

    
    model = Sequential()
    model.add(Dense(350, input_shape = input_shape, activation = "relu"))
    model.add(Dense(50, activation ="relu" ))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    # Glove
    model.compile(loss = 'categorical_crossentropy', optimizer = opt[i], metrics = ['accuracy'])
    model.fit(X_train_g, y_train_g, epochs = 15, batch_size = 200)
    test_results = model.evaluate(X_test_g, y_test_g)
    acc_g.append(test_results[1])






In [ ]:
plt.plot(opt, acc_w2v, color = "red", label = "Word 2 Vec")
plt.plot(opt, acc_g, color = "blue", label = "Glove")
plt.title("Accuracy with Optimizers")
plt.xlabel("Optimizers")
plt.ylabel("Accuracy")
plt.legend(loc = "lower right")
plt.show()


In [ ]:
# Layer 1 for glove and word2vec
acc_w2v = []
acc_g = []

num_layer1 = [100, 200, 300, 400]
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size = 0.20)
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g, y, test_size = 0.20)


for i in range(len(num_layer1)):
        
    model = Sequential()
    model.add(Dense(num_layer1[i], input_shape = input_shape, activation = "relu"))
    model.add(Dense(50, activation = "relu" ))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    
    # Word 2 vec
    model.compile(loss = 'categorical_crossentropy', optimizer = 'RMSprop', metrics = ['accuracy'])
    model.fit(X_train_w2v, y_train_w2v, epochs = 15, batch_size = 200)
    test_results = model.evaluate(X_test_w2v, y_test_w2v)
    acc_w2v.append(test_results[1])
    
    
    model = Sequential()
    model.add(Dense(num_layer1[i], input_shape = input_shape, activation = "relu"))
    model.add(Dense(50, activation = "relu"))
    model.add(Dense(num_classes, activation = 'softmax'))

    # Glove
    model.compile(loss = 'categorical_crossentropy', optimizer = 'RMSprop', metrics = ['accuracy'])
    model.fit(X_train_g, y_train_g, epochs = 15, batch_size = 200)
    test_results = model.evaluate(X_test_g, y_test_g)
    acc_g.append(test_results[1])






In [ ]:

plt.plot(num_layer1, acc_w2v, color = "red", label = "Word 2 Vec")
plt.plot(num_layer1, acc_g, color = "blue", label = "Glove")
plt.title("Accuracy with layer 1 size")
plt.xlabel("Layer 1 sizes")
plt.ylabel("Accuracy")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
# Layer 2 for glove and word2vec
acc_w2v = []
acc_g = []

num_layer2 = [20, 30, 50, 100]
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size = 0.20)
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g, y, test_size = 0.20)


for i in range(len(acc_funcs)):
        
    model = Sequential()
    model.add(Dense(350, input_shape = input_shape, activation = "relu" ))
    model.add(Dense(num_layer2[i], activation = "relu"))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    
    # Word 2 vec
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train_w2v, y_train_w2v, epochs = 15, batch_size = 200)
    test_results = model.evaluate(X_test_w2v, y_test_w2v)
    acc_w2v.append(test_results[1])

    
    model = Sequential()
    model.add(Dense(num_layer1[i], input_shape = input_shape, activation = "relu" ))
    model.add(Dense(50, activation = ))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    # Glove
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train_g, y_train_g, epochs = 15, batch_size = 200)
    test_results = model.evaluate(X_test_g, y_test_g)
    acc_g.append(test_results[1])



In [ ]:

plt.plot(num_layer2, acc_w2v, color = "red", label = "Word 2 Vec")
plt.plot(num_layer2, acc_g, color = "blue", label = "Glove")
plt.title("Accuracy with Layer 2 sizes")
plt.xlabel("Layer 2 sizes")
plt.ylabel("Accuracy")
plt.legend(loc = "lower right")
plt.show()




In [ ]:
# Final Best Model
# Glove outperforms Word2Vec
# Word2Vec
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size = 0.20)

model_w2v = Sequential()
model_w2v.add(Dense(400, input_shape = input_shape, activation = "relu" ))
model_w2v.add(Dense(50, activation = "relu" ))
model_w2v.add(Dense(num_classes, activation = 'softmax'))
    
# Word2vec
model_w2v.compile(loss = 'categorical_crossentropy', optimizer = 'RMSprop', metrics = ['accuracy'])
model_w2v.fit(X_train_w2v, y_train_w2v, epochs = 30, batch_size = 200)
test_results_w2v = model_w2v.evaluate(X_test_w2v, y_test_w2v)
print(f'Test results - Loss: {test_results_w2v[0]} - Accuracy: {test_results_w2v[1]}%')


In [ ]:
# Glove Best Model
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g, y, test_size = 0.20)

model_g = Sequential()
model_g.add(Dense(400, input_shape = input_shape, activation = "relu" ))
model_g.add(Dense(50, activation = "relu" ))
model_g.add(Dense(num_classes, activation = 'softmax'))
    
# Glove
model_g.compile(loss = 'categorical_crossentropy', optimizer = 'RMSprop', metrics = ['accuracy'])
model_g.fit(X_train_g, y_train_g, epochs = 30, batch_size = 200)
test_results_g = model_g.evaluate(X_test_g, y_test_g)
print(f'Test results - Loss: {test_results_g[0]} - Accuracy: {test_results_g[1]}%')


In [ ]:
from prettytable import PrettyTable 


In [ ]:
# Cross Validation
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support




In [ ]:
# Cross Validation
kfold = RepeatedKFold(n_splits = 3, n_repeats = 1)

fold_no = 1
for train, test in kfold.split(X_w2v, y):
    model = buildmodel(input_shape)
    history = model.fit(X_w2v[train], y[train],
              batch_size = 200,
              epochs = 15,
              verbose = 0)
    
    y_pred = model.predict(X_w2v[test])
    y_pred = np.argmax(y_pred, axis = 1)
    y_pred = le.inverse_transform(y_pred)
    y = le.inverse_transform(y)
    print("for Fold Number", fold_no)
    
    d, mx = cf_matrix(y_pred, y[test])
    print("The precision score is")
    precision(d, mx)

    print("The recall score is")
    recall(d, mx)

    print("The F1-score is") 
    f1score(d, mx)

    print("Total precision recall and F1 score is")
    
    print(precision_recall_fscore_support(y_pred, y[test], average = 'weighted'))
    y = le.fit_transform(y)
    fold_no += 1
    



In [ ]:
def buildmodel(input_shape):
    model= Sequential([
        Dense(400, input_shape = input_shape, activation = "relu" ),
        Dense(50, activation = "relu" ),
        Dense(num_classes, activation = 'softmax')
    ])
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'RMSprop', metrics = ['accuracy'])
    return(model)


In [ ]:
# KFold Cross Validation
# Glove
kfold = RepeatedKFold(n_splits = 3, n_repeats = 1)

fold_no = 1
for train, test in kfold.split(X_g, y):
    model = buildmodel(input_shape)
    history = model.fit(X_g[train], y[train],
              batch_size = 200,
              epochs = 15,
              verbose = 0)
    
    y_pred = model.predict(X_g[test])
    y_pred = np.argmax(y_pred, axis = 1)
    y_pred = le.inverse_transform(y_pred)
    y = le.inverse_transform(y)
    print("for Fold Number", fold_no)
    
    d, mx = cf_matrix(y_pred, y[test])
    print("The precision score is")
    precision(d, mx)

    print("The recall score is")
    recall(d, mx)

    print("The F1-score is") 
    f1score(d, mx)

    print("Total precision recall and F1 score is")
    
    print(precision_recall_fscore_support(y_pred, y[test], average = 'weighted'))
    y = le.fit_transform(y)
    fold_no += 1
    



In [ ]:
def cf_matrix(pred_tags,test_tags):
  head = ['PRON','VERB','ADP','DET','ADJ','NOUN','.','ADV','CONJ','X','PRT','NUM', '']
  myTable = PrettyTable(head)
  i=0
  d={}
  for wd in head:
    d[wd]=i
    i+=1
  mx=[]
  for i in range(13):
    mx.append([0]*13)
  print(d)

  for i in range(len(test_tags)):
    gt = test_tags[i]
    pred = pred_tags[i]
    mx[d[gt]][d[pred]] += 1
  for row in mx:
    myTable.add_row(row)
  print(myTable)
  return d,mx



In [ ]:
def recall(d,mx):
  for tag,dex in d.items():
    print("Recall of tag "+tag+" is:")
    if sum(mx[:][dex]) == 0:
        print(0)
    else:
        ans = round(mx[dex][dex]/sum(mx[:][dex]),2)
        print(ans)

def precision(d,mx):
  for tag,dex in d.items():
    print("Precision of tag "+tag+" is:")
    if sum(mx[:][dex]) == 0:
        print(0)
    else:
        ans = round(mx[dex][dex]/sum(mx[dex][:]),2)
        print(ans)

def f1score(d,mx):
  for tag,dex in d.items():
    print("F1 score of tag "+tag+" is:")
    if (sum(mx[dex][:]) == 0) or (sum(mx[:][dex]) == 0):
        print(0)
    else:
        p = round(mx[dex][dex]/sum(mx[dex][:]),2)
        r = round(mx[dex][dex]/sum(mx[:][dex]),2)

        if (p+r) == 0:
            print(0)
        else:
            print(2*p*r/(p+r))